In [ ]:
import deeplake
import cv2
import os
import xml.etree.ElementTree as ET
import struct
from PIL import Image
import re

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from collections import defaultdict

import torch
import torchvision.transforms as transforms
from PIL import Image

int_inf = 9223372036854775807

In [ ]:
# Image preprocessing for resnet 18

# Preprocessing: Resize to 224x224 and stack the grayscale image into 3 channels
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),     # Resize to 224x224
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to 3-channel
    transforms.ToTensor(),             # Convert to tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Normalize
])

# Load and preprocess image
image = Image.open('your_image_path_here').convert('L')  # Open image as grayscale
image_tensor = preprocess(image)  # Preprocess image

# Add batch dimension
image_tensor = image_tensor.unsqueeze(0)  # Add batch size